In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import warnings
warnings.filterwarnings("ignore")
import tensorflow as tf
import numpy as np
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
# init_features, init_labels = np.random.random((100000,100)), np.random.random((100000,1))
graph2 = tf.Graph()
import time

def datagenerator():
    while True:
        time.sleep(1)
        init_features, init_labels = np.random.random((100)), np.random.random((1))
        yield init_features, init_labels

def time_consumer(thresh):
    i = tf.constant(0)
    c = lambda i: tf.less(i, thresh)
    b = lambda i: tf.add(i, 1)
    r = tf.while_loop(c, b, [i], return_same_structure=True)
    return r

with graph2.as_default():
    input_features = tf.placeholder(tf.float32, shape=[None,100] , name = 'input')
    input_labels = tf.placeholder(tf.float32, shape=[None, 1], name = 'labels')
    net = tf.layers.dense(input_features,100,activation=tf.sigmoid)
    net = tf.layers.dense(net,1,activation=tf.sigmoid)
    loss = tf.losses.mean_squared_error(net,input_labels)
    with tf.control_dependencies(time_consumer(int(1e7))):
        train_op = tf.train.AdamOptimizer().minimize(loss)
    init = tf.global_variables_initializer()

In [2]:
%%time
with tf.Session(graph=graph2, config = config) as sess:
    sess.run(init)
    for _ in range(2):
        init_features = []
        init_labels = []
        d = datagenerator()
        for _ in range(10):
            f, l = next(d)
            init_features.append(f)
            init_labels.append(l)
        init_features = np.array(init_features)
        init_labels = np.array(init_labels)
        _, loss_val = sess.run([train_op, loss], feed_dict={input_features: init_features, input_labels:init_labels})
        print(loss_val)

0.052703034
0.105557956
CPU times: user 1min 25s, sys: 413 ms, total: 1min 25s
Wall time: 1min 45s
